In [ ]:
import numpy as np
import scipy.io as scio
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



# Variables in order:
#  CRIM     per capita crime rate by town
#  ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#  INDUS    proportion of non-retail business acres per town
#  CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#  NOX      nitric oxides concentration (parts per 10 million)
#  RM       average number of rooms per dwelling
#  AGE      proportion of owner-occupied units built prior to 1940
#  DIS      weighted distances to five Boston employment centres
#  RAD      index of accessibility to radial highways
#  TAX      full-value property-tax rate per $10,000
#  PTRATIO  pupil-teacher ratio by town
#  B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#  LSTAT    % lower status of the population
#  MEDV     Median value of owner-occupied homes in $1000's
  

In [ ]:
def Gaussian_kernel_matrix(x_train, sigma):# arguments: array and scalar
    l = len(x_train)
    K = np.zeros((l,l))
    for i in range (0,l):
        K[i,i] = 1
        for j in range (i+1,l):
            K[i,j] = np.exp(-np.linalg.norm(x_train[i]-x_train[j])**2/(2*sigma**2))
            K[j,i] = K[i,j]
    K = np.mat(K)
    return K

In [ ]:
def Gaussian_kernel(Xi,Xj,sigma): # arguments: arrays and scalar
    K = np.exp(-np.linalg.norm(Xi - Xj)**2/(2*sigma**2))
    return K

In [ ]:
def alpha_star(K, gamma, y): # arguments: matrix, scalar and array
    l = len(K)
    I = np.mat(np.identity(l))
    a = (K + gamma*l*I).I*y
    return a

In [ ]:
def Cross_validation_parameter_search(X_train, Y_train, gamma, sigma):
    kf = KFold(n_splits=5)
    MSE = np.zeros((len(gamma), len(sigma)))
    best_MSE = 0
    for p1 in range (0, len(gamma)):
        for p2 in range (0, len(sigma)):
            sum_SSE = 0
            for train_index, test_index in kf.split(X_train):
                SSE = 0
                x_train, x_test = X_train[train_index], X_train[test_index]
                y_train, y_test = Y_train[train_index], Y_train[test_index]
                K = Gaussian_kernel_matrix(x_train, sigma[p2])
                a = alpha_star(K, gamma[p1], y_train)
                y_predicted = np.zeros((len(y_test), 1))
                for i in range(0, len(y_test)):
                    for j in range(0, len(x_train)):
                        y_predicted[i, 0] += a[j, 0] * Gaussian_kernel(x_train[j], x_test[i], sigma[p2])
                    SSE += (y_predicted[i, 0] - y_test[i]) ** 2
                sum_SSE += SSE
            MSE[p1, p2] = sum_SSE/(len(Y_train))

            if (p1 == 0 and p2 == 0) or (MSE[p1, p2] < best_MSE):
                best_MSE = MSE[p1, p2]
                gamma_star, sigma_star = gamma[p1], sigma[p2]
    print('Smallest MSE is', np.amin(MSE))
    return gamma_star, sigma_star, MSE

In [ ]:
def split_into_train_test(df):
    train, test = train_test_split(df, test_size=0.33)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)

    # convert data-frames into arrays
    X_train = np.array(train[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                              'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])
    Y_train = np.array(train[['MEDV']])
    X_test = np.array(test[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])
    Y_test = np.array(test[['MEDV']])
    return X_train, Y_train, X_test, Y_test

In [ ]:
# main


data_path="boston.mat"
data = np.array(scio.loadmat(data_path)['boston'])
df = pd.DataFrame(data, columns=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']) 

gamma = []
for i in range (-40, -25):
    gamma.append(2**i)

sigma = []
for i in range (0, 13):
    sigma.append(2**(7+i*0.5))
    
X_train, Y_train, X_test, Y_test = split_into_train_test(df)
gamma_star, sigma_star, MSE_CV = Cross_validation_parameter_search(X_train, Y_train, gamma, sigma)
# gamma_star = 2**(-30)
# sigma_star = 2**9.5

number_of_runs = 20
MSE_train = []
MSE_train_average = 0
MSE_test = []
MSE_test_average = 0
for n in range(0,number_of_runs):
    X_train, Y_train, X_test, Y_test = split_into_train_test(df)
    K = Gaussian_kernel_matrix(X_train, sigma_star)
    a = alpha_star(K, gamma_star, Y_train)
    
    #MSE train
    SSE_train = 0
    y_train = np.zeros((len(Y_train), 1))
    for i in range(0, len(Y_train)):
        for j in range(0, len(X_train)):
            y_train[i, 0] += a[j, 0] * Gaussian_kernel(X_train[j], X_train[i], sigma_star)
        SSE_train += (y_train[i, 0]-Y_train[i])**2
    MSE_train.append(SSE_train/len(Y_train))

    #MSE test
    SSE_test = 0
    y_test = np.zeros((len(Y_test), 1))
    for i in range(0, len(Y_test)):
        for j in range(0, len(X_train)):
            y_test[i, 0] += a[j, 0] * Gaussian_kernel(X_train[j], X_test[i], sigma_star)
        SSE_test += (y_test[i, 0]-Y_test[i])**2
    MSE_test.append(SSE_test/len(Y_test))
    
print(np.mean(MSE_train))    
print(np.mean(MSE_test))
print(np.std(MSE_train))
print(np.std(MSE_test))


In [ ]:
plt.matshow(np.log(MSE_CV))
plt.xlabel("Sigma")
plt.ylabel("Gamma")
plt.show()